In [1]:
import argparse
import collections
import networkx as nx
import tensorflow as tf
import tensorflow.contrib.eager as eager
import my
import objectives
import tf_gnn

/home/gaiyu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
tf.enable_eager_execution(config)

In [3]:
args = argparse.Namespace()
args.depth = 10
args.device = '/cpu:0'
args.graph_tuple = ('soc-Epinions1', 'soc-Slashdot0811', 'soc-Slashdot0902', 'random-50000')
args.lambda_node = 1
args.lambda_edge = 1
args.lambda_entropy = 1
args.n_features = 16
args.n_machines = 10
args.radius = 3

In [4]:
adj_dict = collections.OrderedDict()
for key in args.graph_tuple:
    if key.startswith('random'):
        n = int(key.split('-')[1])
    else:
        g = my.read_edgelist('data/' + key, fill=True)
        adj_dict[key] = nx.adj_matrix(g)

In [5]:
with tf.device(args.device):
    for key, adj in adj_dict.items():
        objective = objectives.Objective(adj, args.n_machines, args.lambda_node, args.lambda_edge, args.lambda_entropy)
        gnn = tf_gnn.GNN((1,) + (args.n_features,) * args.depth, args.n_machines, adj, args.radius)
        
        ckpt = eager.Checkpoint(model=gnn)
        ckpt.restore('./models/gnn-1')
        
        x = tf.constant(adj.sum(1), dtype=tf.float32)
        x -= tf.reduce_mean(x)
        x /= tf.sqrt(tf.reduce_mean(tf.square(x)))
        
        z = gnn(x, training=True)
        r, b_node, b_edge, entropy = objective(z)
        
        print('[%s]%f %f %f %f' % (key, r, b_node, b_edge, entropy))

[soc-Epinions1]2.686759 -2.242767 -1.802942 0.797796
[soc-Slashdot0811]4.848514 -2.284688 -2.097455 1.086525
[soc-Slashdot0902]4.846651 -2.286368 -2.108891 1.096381
